In [1]:

import pandas as pd
import numpy as np
import main as fca
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics
from importlib import reload
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import lazy_pipeline as lpipe
import folium
from folium.plugins import HeatMap
import plotly.express as px
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,plot_roc_curve,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline
pd.set_option('display.max_columns', 32)

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
column_names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
        'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size',
        'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']
data = 'breast-cancer-wisconsin.data'
df = pd.read_csv(data, names=column_names)
df.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [3]:
(df['Bare Nuclei'] == '?').sum() #checking for rows with ? in it

16

In [4]:
df = df[df['Bare Nuclei'] != '?'] #dropping rows with ? present in it

In [5]:
df['Bare Nuclei'] = df['Bare Nuclei'].astype('int64') #we have all the features in numerical value.

In [6]:
df = df.drop(['Sample code number'], axis=1) #since we don't need code number so dropping it from the table
#df = df.select_dtypes(exclude=['object'])
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [7]:
df.loc[df['Class'] == 2, 'Class'] = 0 #updating target labels
df.loc[df['Class'] == 4, 'Class'] = 1

In [8]:
df = df.dropna()
df.head()
cols = list(df.columns)
cols.remove('Class')
X = df.loc[:,cols]
y = df.Class
X.shape,y.shape

((683, 9), (683,))

In [9]:
X = X.sample(frac=1, random_state=42)  # Shuffle X
y = y[X.index]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [11]:
import main as fca

In [12]:
fca.fitClassifier(X_train, y_train)

In [13]:
y_pred = fca.predict(X_test)

In [14]:
results = {'accuracy': [], 'precision': [], 'recall': []}
results['accuracy'].append(metrics.accuracy_score(y_test, y_pred))
results['precision'].append(metrics.precision_score(y_test, y_pred))
results['recall'].append(metrics.recall_score(y_test, y_pred))

print('Accuracy:', pd.DataFrame(results)['accuracy'])

Accuracy: 0    0.892683
Name: accuracy, dtype: float64


In [15]:

classifiers = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    ]

log_cols=["Classifier", "Accuracy", "Precision", "Recall"]
log = pd.DataFrame(columns=log_cols)


In [16]:
for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    
    train_predictions = clf.predict(X_test)
    prec = precision_score(y_test, train_predictions)
    
    
    train_predictions = clf.predict(X_test)
    rec = recall_score(y_test, train_predictions)
      
    
    train_predictions = clf.predict_proba(X_test)
    
    
    log_entry = pd.DataFrame([[name, acc, prec, rec]], columns=log_cols)
    log = log.append(log_entry)
    
print("="*30)

In [17]:
print(log)

                   Classifier  Accuracy  Precision    Recall
0      DecisionTreeClassifier  0.951220   0.971429  0.894737
0      RandomForestClassifier  0.975610   0.986301  0.947368
0          AdaBoostClassifier  0.951220   0.971429  0.894737
0  GradientBoostingClassifier  0.970732   0.986111  0.934211


In [18]:
f_acc,f_pre,f_rec = results['accuracy'][0],results['precision'][0],results['recall'][0]
fcaa_row = {'Classifier': 'FCA', 'Accuracy': f_acc, 'Precision': f_pre,'Recall':f_rec}
log = log.append(fcaa_row, ignore_index = True)
  
display(log)

,Classifier,Accuracy,Precision,Recall
0,DecisionTreeClassifier,0.951220,0.971429,0.894737
1,RandomForestClassifier,0.975610,0.986301,0.947368
2,AdaBoostClassifier,0.951220,0.971429,0.894737
3,GradientBoostingClassifier,0.970732,0.986111,0.934211
4,FCA,0.892683,1.000000,0.710526


In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

log.sort_values(by = 'Accuracy', ascending = False)
fig = px.bar(data_frame = log, x = 'Accuracy', y = 'Classifier', color = 'Accuracy', template = 'ggplot2', title = 'Models Comparison')
fig.show()